# Final Project

Analyze the collected Twitter data with OpenAI and store the results in a MongoDB database. The analyses include:

- Sentiment analysis
- Language translation
- Identify emotions
- Extract entities
- Summarize

## Install Python libraries.

- pymongo: manage the MongoDB database
- openai: call the OpenAI APIs.

In [1]:
pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 55.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


## Secrete Manager Function

In [3]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Import Python Libraries and Credentials  

In [4]:
import pymongo
from pymongo import MongoClient
import json
from pprint import pprint
from tqdm.auto import tqdm
import re

openai_api_key  = get_secret('openai')['api_key']

mongodb_connect = get_secret('mongodb')['connection_string']

## Connect to the MongoDB cluster

In [5]:
mongo_client = MongoClient(mongodb_connect)
db = mongo_client.demo # use or create a database named demo
job_collection = db.job_collection #use or create a collection named tweet_collection
#job_collection.create_index([("tweet.id", pymongo.ASCENDING)],unique = True) # make sure the collected tweets are unique

## Extract Twitter Data

Filter the Tweets you are interested in. You can use MongoDB Compass to help you write the queries.

In [18]:
filter={

    
}
project={
    'QualificationSummary': 1, 
    'PositionID': 1
}
#rename the client to mongo_client
result = mongo_client['demo']['job_collection'].find(
  filter=filter,
  projection=project
)

Save the extracted Tweets into the ```tweet_data``` list. Remove URLs and new lines to save the tokens.

In [19]:
job_data = []
#url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
for job in result:
   # text_without_urls = re.sub(url_pattern, '', tweet['tweet']['text'])
   job_data.append({'position_ID':job['PositionID'], 'summary':job['QualificationSummary']})

In [22]:
print('Number of jobs: ',len(job_data))
print(job_data)

Number of jobs:  211
[{'position_ID': 'DE-12563628-25-IA', 'summary': 'GS-11 Level: You may qualify at this grade level by meeting one (1) of the following below: Specialized Experience GS-11 To qualify for the GS-11 grade level, you must possess the specialized experience at the GS-9 grade level in federal service or equivalent private/public sector experience. For this position, specialized experience is defined as: Having knowledge and practical experience in the oversight of DHS Intelligence, counterintelligence and intelligence-related activities. Knowledge of the National Security Act of 1947, as amended; E.O. 12333, and other authorities addressing intelligence collection, processing and analysis. Knowledge of Congressional IO procedures; DHS policy; and applicable case law. NOTE: Your resume must explicitly indicate how you meet this requirement, otherwise you will be found ineligible. -OR- Substitution of Education for Experience You may substitute education for experience at 

## Set up OpenAI API

Load the OpenAI API key and set the API parameters.

- Model type: usegpt-4o by default, and you choose any [availabel models](https://platform.openai.com/docs/models).
- Token estimate: 100 tokens ~= 75 words in English. Total token usage = tokens in the prompt + tokens in the completion. You can get a more accurate estimate at [Tokenier](https://platform.openai.com/tokenizer).
- Temperature: Lower temperatures produce more consistent outputs, while higher values generate more diverse and creative results. 

A help function, ```openai_help```, is created to pass the prompt.

In [21]:
from openai import OpenAI
client = OpenAI(api_key=openai_api_key)
model="gpt-4o"
temperature=0

def openai_help(prompt, model=model, temperature =temperature ):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature

    )
    return response.choices[0].message.content

## Extract entities

Extract person and organization names from each tweet and save the result to the MongoDB database.

In [ ]:
for job in tqdm(job_data):
    
  
    prompt = f"""
    extract commmon skills from the job summary,
    summary text: {job['summary']},
    format the response as a JSON object in a list,
    if no skills is not presented, use "Unknown" in the list.
    Do not wrap the JSON codes in JSON markers
   
    """
#     print(prompt)
    
    extract_result =openai_help(prompt)
#        print(extract_result)

    job_collection.update_one(
                {'PositionID':job['position_ID']},
                {"$set":{'skills':json.loads(extract_result)}}
                )


  0%|          | 0/211 [00:00<?, ?it/s]

In [15]:
mongo_client.close()